# 파이토치로 구현하는 신경망

파이토치를 이용하여 가장 기본적인 신경망을 만들어봅니다.
 * [개념] 텐서와 Autograd
 * [프로젝트 1] 텐서와 Autograd
 * [프로젝트 2] 신경망 모델 구현하기
 * [프로젝트 2] 토치비전과 토치텍스트로 데이터셋 다루기

파이토치는 기본적인 수학 계산용 라이브러리를 바탕으로 그 위에 머신러닝에 필요한 그래프 형태의 계산방식을 추가 시킨 라이브러리 입니다. 물론 파이토치의 바탕이 되는 계산 라이브러리에 대한 깊은 지식이 없더라도 파이토치를 이용해 머신러닝 모델을 구현하는데 그리 큰 문제는 없습니다.
하지만 파이썬 개발자들에게 편리하도록 설계 되었더라도 수리적 계산이 많이 들어가는 머신러닝의 특성 때문에 파이토치의 자료구조는 기존 파이썬의 자료구조와는 사뭇 다릅니다. 
파이토치의 가장 기본적인 자료구조인 텐서(Tensor) 가 그 대표적인 예 인데요,이번 장에선 이 텐서와 텐서를 이용한 연산, 그리고  Autograd 등의 기능을 배워 보겠습니다. 더불어 이들을 이용해 기본적인 신경망 모델을 구현 해 보고 저장, 재사용 하는 방법까지 배워 보겠습니다.

## 프로젝트 1. 텐서와 Autograd

프로그래밍 언어를 배울 때와 마찬가지로, 파이토치 또한 직접 코딩을 하면서 배우는 것이 가장 효율적인 방법이라고 생각합니다. 간단한 파이토치 코드 예제를 같이 코딩하면서 파이토치에 대해 공부 해 보겠습니다.

### 텐서 다루기 기본: 차원(Rank)과 모양(Shpae)

가장 먼저 파이토치를 임포트 합니다.

```python
import torch
```

텐서(Tensor)는 파이토치에서 다양한 수식을 계산하기 위한 가장 기본적인 자료구조 입니다. 흔히 수학에서 말하는 벡터나 행렬 과 같은 개념이며, 숫자들을 특정한 모양으로 배열 한 것입니다. 그럼 간단한 텐서를 만들어 보겠습니다. 

```python
x = torch.tensor([[1,2,3], [4,5,6], [7,8,9]])
print(x)
```
	
위 코드는 다음과 같은 결과를 출력합니다.

```
tensor([[1,  2,  3], [ 4,  5,  6], [7, 8, 9]])
```

즉 x는 1부터 9까지의 숫자를 가로 3줄, 세로 3줄의 모양을 지니도록 배열한 텐서입니다. 그리고 가로와 세로 두 차원으로만 이루어져 있는 2차원 텐서라고 할 수 있습니다.
이처럼 텐서는 랭크(Rank) 과 모양(Shape) 이라는 개념을 갖고 있습니다. 텐서의 랭크가 0이면 스케일러(Scaler), 1이면 벡터(Vector), 2면 행렬(Matrix), 3이상이면 n 랭크 텐서 라고 부릅니다.

```python
1 -> 스케일러, 모양은 []
[1,2,3] -> 벡터, 모양은 [3]
[[1,2,3]] -> 행렬, 모양은 [1,3]
```

텐서의 랭크과 모양은 size() 함수 혹은 shape 키워드를 통해 확인 할 수 있습니다.

```python
print(x.size())
print(x.shape)
```

```python
torch.Size([3, 3])
torch.Size([3, 3])
```
 
unsqueeze(), squeeze(), 그리고 view() 함수를 통해 우리는 인위적으로 텐서의 랭크와 모양을 바꿔 줄 수도 있습니다.
먼저 unsqueeze() 함수를 통해 텐서 x의 랭크를 늘려 보겠습니다.

```python
x = torch.unsqueeze(x, 0)
print(x)
print(x.shape)
```

위 코드는 텐서 모양의 첫번째(0 번째) 자리에 1 이라는 차원값을 인위적으로 추가 시켜 [3,3] 모양의 랭크 2 텐서를 [1,3,3] 모양의 랭크 3 텐서로 변경시킵니다. 랭크는 늘어나도, 텐서 속 원소의 수는 유지됩니다.

```python
tensor([[[ 1,  2,  3], [ 4,  5,  6], [ 7,  8,  9]]])
torch.Size([1, 3, 3])
```

squeeze() 함수를 이용하면 텐서의 랭크 중 크기가 1인 랭크를 삭제하여 다시 랭크 2 텐서로 되돌릴 수 있습니다.  [1, 3, 3] 모양을 가진 텐서 x 를 다시 [3,3] 모양으로 되돌려 보겠습니다.

```python
x = torch.squeeze(x)
print(x)
print(x.shape)
```

```python
tensor([[1,  2,  3], [ 4,  5,  6], [ 7,  8,  9]])
torch.Size([3, 3])	#[3,3] 모양의 랭크 2 텐서
```

x 는 이제 랭크 2의 텐서가 되었지만 이번에도 역시 텐서 속의 총 숫자 수는 계속 9로 영향을 받지 않았습니다.
view()함수를 이용하면 위와 같은 작업을 더 쉽게 할 수 있을 뿐만 아니라, 직접 텐서의 모양을 바꿔 줄 수도 있습니다. 랭크 2의 [3,3] 모양을 한 x 를 랭크 1의 [1,9] 모양으로 바꿔 보겠습니다.
 
```python
x = x.view(9)
print(x)
print(x.shape)
```

```python
tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9])
torch.Size([9])
```

이제 텐서 x는 [9] 모양을 한 랭크 1 텐서가 되었습니다. 
이처럼 squeeze(), unsqueeze(), view() 함수는 텐서 속 원소의 수를 그대로 유지하면서 텐서의 모양과 차원을 조절합니다. 말인즉슨, view() 함수에 잘못된 모양을 입력하면 함수는 실행 될 수 없습니다.
예를 들어 view 함수를 이용해 x 의 모양을 [2, 4] 가 되도록 만들어 보겠습니다. 

```python
x = x.view(2,4)
Print(x)
```

코드를 실행시키면 다음과 같은 에러 메시지를 보게 됩니다.

```python
Traceback (most recent call last):
  File "tensor_autograd.py", line 12, in <module>
    x = x.view(2,4)
RuntimeError: invalid argument 2: size '[2 x 4]' is invalid for input with 9 elements at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/THStorage.c:41
```

이처럼 원소가 9 개인 텐서를 2 X 4, 즉 8 개 의 원소를 가진 텐서로 바꿔주는것은 불가능합니다.

### 전체 코드
```python
import torch

x = torch.tensor([[1,2,3], [4,5,6], [7,8,9]])

print(x)
print(x.size())
print(x.shape)

x = torch.unsqueeze(x, 0)
print(x)
print(x.shape)

x = torch.squeeze(x)
print(x)
print(x.shape)

x = x.view(9)
print(x)
print(x.shape)

x = x.view(2,4)
Print(x)
```

### 텐서를 이용한 연산과 행렬곱

딥러닝을 하는데 수준 높은 수학적 지식이 필요하지는 않습니다. 하지만 기본적으로 행렬과 행렬곱은 모든 딥러닝 알고리즘에 사용되므로 꼭 짚고 넘어가면 좋습니다. 앞서 말씀드렸듯이 행렬은 2차원 텐서와 같은 개념입니다. 숫자들을 네모꼴로 배열한 것으로, 네모꼴의 높이를 행, 넓이를 열 이라고 합니다. 만약 A, B 라는 두 행렬을 가지고 행렬곱을 할 시 다음과 같은 조건이 성립해야 합니다.

```
A 의 열 수와 B 의 행 수는 같아야 한다.
행렬곱 A X B 를 계산한 행렬은 A의 행 개수, 그리고  B 의 열 개수를 가지게 된다.
```

<img src="./images/mm.png" width="200">

그러면 직접 파이토치를 이용해 행렬곱을 구현해 보겠습니다. 우선 행렬곱에 사용될 두 행렬을 정의합니다.

```python
w = torch.randn(5,3, dtype = torch.float)
x = torch.tensor([[1.0,2.0], [3.0,4.0], [5.0,6.0]])
```

randn() 함수는 정규분포(Normal Distribution)에서 무작위하게 float32 형의 숫자들을 선택해 w 라는 텐서를 채워넣습니다. 그리고 텐서 x에는 직접 float 형의 원소들을 집어넣어 주었습니다.
행렬곱 외에도 다른 행렬 연산에 쓰일 b 라는 텐서도 추가로 정의해 보겠습니다.

```python
b = torch.randn(5,2, dtype = torch.float)
```

행렬곱을 하려면 torch.mm() 함수를 사용하면 됩니다.

```python
wx = torch.mm(w,x) # w의 행은 5, x의 열은 2 즉 [5,2] 의 형태
```

이 wx 행렬의 원소들에 b 행렬의 원소들을 더해 보겠습니다.

result = wx + b

위의 텐서들을 출력시켜 보면, x 는 [5, 3], w 는 [3, 2], 그리고 나머지 텐서는 [5, 2] 형태를 띄고 있음을 확인 할 수 있습니다.

#### 전체 코드

```python
import torch

w = torch.randn(5,3, dtype = torch.float)
x = torch.tensor([[1.0,2.0], [3.0,4.0], [5.0,6.0]])
b = torch.randn(5,2, dtype = torch.float)
wx = torch.mm(w,x)
result = wx + b

print(x)
print(w)
print(b)
print(wx)
print(result)
```

### Autograd 

Autograd 는 머신러닝에 필수적인 최적화 알고리즘인 ***경사 하강법(Gradient Descent)*** 에 관련된 기능을 제공합니다. 처음 머신러닝을 접하시는 분들은 이 알고리즘이 무엇인지, 그리고 어떻게 머신러닝에 관련되 있는지 몰라 고개를 갸웃거리실 수도 있습니다. 그런 분들을 위해 이번에는 직접 코드를 짜보기에 앞서 머신러닝의 학습 원리에 대하여 조금 더 깊게 배워보고 이 알고리즘이 어떻게 머신러닝에 사용되는지 알아보겠습니다.

앞 장에서 배웠듯 머신러닝 모델은 입력된 데이터를 기반으로 학습합니다. 다시말해 아직 충분한 데이터를 입력받지 못하거나 학습을 아직 끝내지 않은 모델은 입력된 데이터에 대해 잘못된 결과를 출력하게 됩니다.
이처럼 입력 데이터에 대해 정해진 답(Ground Truth) 과 머신러닝 모델이 낸 답의 차이를 산술적으로 표현한 것을 ***거리(Distance)*** 라고 합니다. 그리고 학습에 이용되는 데이터들을 가지고 계산된 거리들의 평균을 ***오차(loss)*** 라고 일컫습니다. 즉, 오차 값이 작은 머신러닝 모델일수록 주어진 데이터에 대해 더 정확한 답을 낸다고 볼수 있습니다.

오차값을 최소화 하는데는 여러 알고리즘이 쓰이고 있지만, 가장 유명하고 많이 쓰이는 알고리즘은 바로 전 언급한 경사하강법 이라는 알고리즘입니다. 오차를 수학적 함수로 표현한 후, 오차 함수의 기울기를 구해 오차의 최소값이 있는 곳의 방향을 찾아내는 알고리즘이죠. 간단한 경사하강법은 Numpy와 같은 라이브러리 만으로도 직접 구현이 가능합니다만 복잡한 인공신경망 모델에선 어렵고 머리아픈 미분식의 구현과 계산을 여러번 해 주어야 합니다. 다행히도 파이토치의 Autograd는 이름 그대로 파이토치 라이브러리 내에서 미분과 같은 수학 계산들을 자동화 시켜 우리로부터 직접 경사하강법을 구현하는 수고를 덜어줍니다.
그럼 Autograd를 어떻게 사용하는지 같이 공부해 보겠습니다.

우선 값이 1인 w 라는 0차원 스케일러 텐서를 만들어 보겠습니다. 방금 전 설명에서 Autograd가 미분 계산을 자동화 해준다고 설명했는데요, 쉽게 말하면 w 가 변수로 들어가는 수식을 w로 미분하고 기울기를 계산해 준다고 이해하면 됩니다. 이를 위해선 텐서 w의 requires_grad 키워드를 True로 설정해야 합니다.
아주 쉬운 예를 통해 간단한 미분식을 계산 해 보겠습니다.

```python
w = torch.tensor(1, requires_grad=True)
a = w*3
```

a 라는 수식을 w 곱하기 3이라고 정의했습니다. 즉 이 식의 w에 대한 기울기는 3 입니다. backward() 함수를 이용하면 이 수식의 기울기를 구할 수 있습니다.

```python
a.backward()
print(w.grad)
```

예상대로, 위 코드는 다음과 같이 3 이라는 결과를 출력합니다.

```python
tensor(3)
```

간단한 미분식과 기울기 계산을 해 봤으니, 이번엔 조금 더 복잡한 미분식 계산을 해 보겠습니다.

```python
w = torch.tensor(1, requires_grad=True)
a = w*3
l = a*2
```

위의 l은 텐서 a의 모든 값을 제곱한 텐서 입니다.
텐서 w에 3을 곱해 a를 만들었고, 또 a를 제곱하여 l을 만들었습니다.
이를 수식으로 표현하면 다음과 같습니다.

```python
l = 2*a
a = 3*w
그러므로
l = 2*(3*w) = 6w
```

이러한 l을 w로 미분하려면 연쇄법칙(Chain Rule)을 이용하여 l을 a와 w로 차례대로 미분해 줘야합니다.

```python
l.backward()
print('l을 w로 미분한 값은 ', w.grad)
```

위의 코드를 실행하면 다음과 같은 결과를 확인 하실 수 있습니다.

```python
l을 w로 미분한 값은  tensor(6)
```

backward() 함수는 l을 a로 미분한 후, 그 값을 a를 w로 미분한 값에 곱해줘 w.grad 를 계산했습니다.
여러 겹의 행렬곱을 하는 인공신경망이 경사 하강법을 할때는 위처럼 여러 겹의 미분식을 해야합니다.
이렇게 연쇄법칙을 사용하여 경사 하강법을 하는 딥러닝 특유의 알고리즘이 바로 그 유명한
***역전파 알고리즘(Backpropagation Algorithm)*** 입니다.

역전파 알고리즘은 딥러닝에 있어 가장 자주 쓰이는 알고리즘 이지만 직접 구현하는데에는 복잡한 코드와 수학적 지식이 필요합니다.
다행히 파이토치는 역전파 알고리즘 기법을 제공해주기 때문에 우리가 직접 역전파 알고리즘을 구현할 일은 없습니다만, 아주 중요한 알고리즘이므로
딥러닝을 좀 더 깊게 공부하고자 하신다면 꼭 자세히 공부하는걸 권하고 싶습니다.

#### 전체 코드

```python
import torch

w = torch.tensor(1, requires_grad=True)
a = w*3
l = a*2

l.backward()
print('l을 w로 미분한 값은 ', w.grad)
```

## 프로젝트 2. 신경망 모델 구현하기

이번 장에서는 지금까지 배워 온 개념들을 토대로 간단한 신경망을 함께 구현해 보겠습니다. 지금까지 내용과는 달리, 이번 장에는 딥러닝에 핵심적인 내용을 조금 더 깊게, 그리고 이론적으로 설명하여 처음 딥러닝을 접하는 분들에게는 다소 어려울 수도 있습니다. 하지만 설명을 읽어가며 함께 코딩을 해 보면 어느새 딥러닝을 코딩하는데 익숙해 질 것입니다.

### 딥러닝과 인공신경망

이름에서부터 알 수 있듯이 인공신경망은 인간의 뇌, 혹은 신경계의 작동 방식에서 그 영감을 받았습니다. 신경계가 작동을 하기 위해선 가장 먼저 눈이나 혀 같은 감각 기관을 통해 자극을 입력 받아야 합니다. 이런 자극이 첫번째 신경세포로 전달되고, 이 신경세포는 자극을 처리해 다른 신경세포로 전달합니다. 이러한 자극 처리와 전달 과정을 여러번 반복하다 보면 인간의 신경계는 수많은 자극을 인지하고 그에 따라 다른 반응을 하게됩니다. 그러다 언젠가는 맛을 판별하거나 손가락을 움직이는 등 다양하고 복잡한 작업을 할수 있게 됩니다.

자극을 텐서의 형태로 입력받는 인공신경망에선 이러한 자극의 입력과 전달과정이 행렬곱 과 활성화 함수 라는 수학적 연산으로 표현됩니다.
실제 인간의 신경세포가 자극을 전달하기 전에 입력받은 자극에 여러 화학적 가공처리를 가하듯 인공신경망도 입력된 텐서에 특정한 수학적 연산을 실행합니다. 바로 ***가중치(Weight)*** 라고 하는 랜덤한 텐서를 행렬곱 시켜주는 것이죠.
그리고 이 행렬곱의 결과는 ***활성화 함수(Activation Function)*** 를 거쳐 결과값을 산출하게 됩니다. 이 결과값이 곧 인접한 다른 신경세포로 전달되는 자극이라고 보시면 됩니다.
자극의 처리와 전달, 이러한 과정을 몇겹에 싸여 반복한 후 마지막 결과값 만들어 내는 것이 인공신경망의 기본적인 작동원리입니다.

### 간단한 분류 모델 구현하기

이번 장에서는 인공신경망을 이용해 간단한 분류 모델을 함께 구현해 보겠습니다. 하지만 처음 인공신경망과 머신러닝을 접하는 분들을 위해 이미지 같은 고차원의 복잡한 데이터가 아닌 간단한 2차원의 데이터를 이용하겠습니다. 첫번째 인공신경망을 구현하는 만큼, 이번 프로젝트의 코드는 조금 새롭고 생소한 개념을 다소 포함하고 있습니다. 그러므로 꼭 설명을 자세하게 읽어 보고 코딩해 보시기 바랍니다.

우선 파이토치와 그 외 다른 라이브러리들을 임포트합니다. Numpy는 유명한 수치 해석용 라이브러리 입니다. 행렬과 벡터를 이용한 연산을 하는데 아주 유용한 라이브러리며, 파이토치도 이 넘파이를 기반으로 개발되었을 정도로 긴밀하게 이용됩니다. 이번 프로젝트에서는 인공신경망 학습을 위한 데이터를 만드는데 넘파이와 sklearn 라이브러리를 이용하여 생성하겠습니다. 마지막으로 임포트 되어지는 matplotlib 라이브러리는 데이터를 시각화 하는데 있어 유용한 툴 입니다. 학습데이터가 어떠한 패턴을 보이며 분포되어 있는지 확인하기 위해 matplotlib 을 이용하겠습니다.

```python
import torch
import numpy
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plot
import torch.nn.functional as F
```

인공신경망을 구현하기 전 인공신경망의 학습과 평가를 위한 데이터셋을 만들어 줍니다.
밑의 코드에서 x_tra 와 y_tra 라고 정의된 실험데이터는 직접 인공신경망을 학습시키는데 쓰이는 데이터 입니다. 반대로 x_tes 와 y_tes 라고 정의된 데이터는 직접 신경망을 학습시키는데는 쓰이지 않지만 학습이 끝난 신경망의 성능을 평가하고 실험하는데 쓰일 데이터 셋입니다.

```python
n_dim = 2
x_tra, y_tra = make_blobs(n_samples=80, n_features=n_dim, centers=[[1,1],[-1,-1],[1,-1],[-1,1]], shuffle=True, cluster_std=0.3)
x_tes, y_tes = make_blobs(n_samples=20, n_features=n_dim, centers=[[1,1],[-1,-1],[1,-1],[-1,1]], shuffle=True, cluster_std=0.3)
```

make_blobs() 함수를 이용하여 데이터를 2차원 벡터의 형태로 만들어 주었습니다.
학습데이터(Training Data Set)에는 80개, 실험데이터(Test Data Set)에는 20개의 2차원 벡터 형태의 데이터가 있는 것을 확인하실 수 있습니다.
데이터를 만든 후, 데이터에 해당하는 정답인 ‘레이블’ 을 달아줍니다. label_map 이라는 간단한 함수를 구현해 데이터가 [-1, -1] 혹은 [1, 1] 주위에 있으면 0 이라는 레이블을 달아 줬습니다. 반대로 [1, -1] 혹은 [-1, 1] 주위에 위치해 있으면 1 이라는 레이블을 달아 줬습니다.

```python
def label_map(y_, from_, to_):
    y = numpy.copy(y_)
    for f in from_:
        y[y_ == f] = to_
    return y

y_tra = label_map(y_tra, [0, 1], 0)
y_tra = label_map(y_tra, [2, 3], 1)
y_tes = label_map(y_tes, [0, 1], 0)
y_tes = label_map(y_tes, [2, 3], 1)
```

데이터가 제대로 만들어 졌는지, 그리고 제대로 레이블링이 되었는지 확인하기 위해 matplotlib 을 이용해 데이터를 시각화 해 보겠습니다.

```python
def vis_data(x,y = None, c = 'r'):
	if y is None:
		y = [None] * len(x)
	for x_, y_ in zip(x,y):
		if y_ is None:
			plot.plot(x_[0], x_[1], '*',markerfacecolor='none', markeredgecolor=c)
		else:
			plot.plot(x_[0], x_[1], c+'o' if y_ == 0 else c+'+')

plot.figure()
vis_data(x_tra, y_tra, c='r')
plot.show()
```

레이블이 0 인 학습 데이터는 점으로, 1인 데이터는 십자가로 표시했습니다.

<img src="./images/data_distribution.png" width="200">

마지막으로 신경망을 구현 하기 전, 위에서 정의한 데이터들을 넘파이 리스트가 아닌 파이토치 텐서로 바꿔줍니다.

```python
x_tra = torch.FloatTensor(x_tra)
x_tes = torch.FloatTensor(x_tes)
y_tra = torch.FloatTensor(y_tra)
y_tes = torch.FloatTensor(y_tes)
```

이제 데이터를 준비했으니 본격적으로 신경망 모델을 구현해 보겠습니다. 
파이토치에서 인공신경망은 아래와 같이 신경망 모듈(Neural Network Module)을 상속받는 파이썬 객체 로 나타낼 수 있습니다.

```python
class Feed_forward_nn(torch.nn.Module):
```

인공신경망의 구조와 동작을 정의하는 컨스트럭터/이니셜라이져(Constructor/Initializer) 를 모델 클래스 안에 정의해 보겠습니다.

```python
		def __init__(self, input_size, hidden_size):
```

__init()__ 함수는 파이썬 객체지향 프로그래밍에서 객체가 생성될 때 객체에 내포된 값을 설정 해 주는 함수이며, 객체가 생성 될 때 자동적으로 호출됩니다. 이번 예제에서는 학습/실험 데이터의 차원인 input_size 라는 변수와 hidden_size 라는 변수를 __init()__ 함수를 통해 설정하도록 구현했습니다.
input_size 는 신경망에 입력되는 데이터들의 차원입니다. 
2차원 데이터를 입력받는 모델을 구현할 것이므로 input_size는 2라고 정의됩니다.
[1,2] 사이즈의 입력데이터가 [2,5] 모양을 가진 가중치 텐서와 행렬곱 해 [1,5] 모양의 텐서가 만들어지듯이, 신경망에 입력된 데이터는 신경망 속의 가중치와 활성화 함수를 거치며 차원을 변화시킵니다. 이렇게 중간에 변화된 차원값을 hidden_size 라고 부르겠습니다.

```python
			super(Feed_forward_nn, self).__init__()
			self.input_size = input_size
			self.hidden_size  = hidden_size
```

다음은 입력된 데이터가 인공신경망을 통과하면서 거치는 연산들을 정의해 주겠습니다.

```python
			self.linear_1 = torch.nn.Linear(self.input_size, self.hidden_size)
			self.relu = torch.nn.ReLU()
			self.linear_2 = torch.nn.Linear(self.hidden_size, 1)
			self.sigmoid = torch.nn.Sigmoid()
```

linear_1 함수는 앞서 여러번 반복해 설명드렸던 행렬곱을 하는 함수입니다. [input_size, hidden_size] 사이즈의 가중치를 입력 데이터에 행렬곱 시켜 [1,hidden_size] 꼴의 텐서를 리턴합니다. 이 때 리턴된 값은 torch.nn.ReLU() 라는 활성화 함수를 거치게 됩니다. ReLU 는 입력값이 0보다 작으면 0을, 0보다 크면 입력값을 그대로 출력합니다. 예를 들어 텐서 [-1, 1, 3, -5]가 ReLU 를 거치면 텐서 [0, 1, 3, 0]가 리턴됩니다.

<img src="./images/ReLU.png" width="200">

ReLU 를 통과한 텐서는 다시 한번 linear_2 로 정의된 행렬곱을 거쳐 [1,1] 꼴을 지니게 됩니다. 마지막으로 이 텐서는 sigmoid 활성화 함수에 입력됩니다.  Sigmoid 는 입력된 학습데이터가 레이블 1에 해당할 확률값을 리턴하는 함수로써, 머신러닝과 딥러닝에서 가장 중요한 활성화 함수 입니다.

<img src="./images/sigmoid.png" width="200">

위의 그림처럼 sigmoid 함수는 0과 1 사이의 값을 리턴합니다. 
다음으로 __init__() 함수에서 정의된 동작들을 차례대로 실행하는 forward() 함수를 구현합니다.

```python
		def forward(self, input_tensor):
			linear1 = self.linear_1(input_tensor)
			relu = self.relu(linear1)
			linear2 = self.linear_2(relu)
			output = self.sigmoid(linear2)
			return output
```

이로써 인공신경망을 구현이 끝났습니다. 이제 실제로 신경망 객체를 생성하고 학습에 필요한 여러 변수와 알고리즘을 정의하겠습니다.

```python
model = Feed_forward_nn(2, 5)
learning_rate = 0.03
criterion = torch.nn.BCELoss()
```

input_size 를 2로, hidden_size 를 5 로 설정한 신경망 객체를 만들었습니다. learning_rate 은 ‘얼마나 급하게 학습하는가’ 를 설정하는 값입니다. 값이 너무 크면 오차함수의 최소점을 찾지 못하고 지나치게 되고, 값이 너무 작으면 학습속도가 느려집니다.
러닝레이트를 설정했으면 그 다음으로는 오차함수를 만들어야 합니다. 물론 직접 오차함수를 코딩 할 수도 있지만 이는 매우 까다롭고 귀찮은 일입니다. 다행히도 파이토치는 여러 오차함수를 미리 구현해서 바로 사용 할 수 있도록 해놓았습니다. 이번에 우리는 파이토치가 제공해 주는 이진교차 엔트로피(Binary Cross Entropy) 라는 오차함수를 사용하겠습니다.

epochs는 학습데이터를 총 몇번 반복
동안 오차를 구하고 그 최소점으로 이동 할지 결정해줍니다. 
마지막 변수 optimizer 는 최적화 알고리즘입니다. 최적화 알고리즘 에는 여러 종류가 있고 상황에 따라 다른 알고리즘을 사용합니다. 이번 예제를 통해 처음으로 인공신경망을 구현하는 분들을 위해 그중에서도 가장 기본적인 알고리즘인 스토카스틱 경사 하강법(Stochastic Gradient Descent)을 사용하겠습니다.

```python
epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
```

학습을 시작하기 전 정말 마지막으로 아무 학습도 하지 않은 모델의 성능을 시험해 보겠습니다.

```python
model.eval()
test_loss_before =  criterion(model(x_tes).squeeze(), y_tes)
print('Before Training, test loss is ', test_loss_before.item())
```

위 코드는 아래와 같은 결과를 출력합니다.

```
Before Training, test loss is 0.7301096916198730
```

오차값이 0.73 이 나왔습니다. 이정도의 오차를 가진 모델은 사실상 분류하는 능력이 없다고 봐도 무방합니다.
자, 이제 드디어 인공신경망을 학습시켜 퍼포먼스를 향상시켜 보겠습니다.

우선 epoch을 반복해주는 for loop 을 만들어 줍니다.

```python
for epoch in range(epochs):
```

모델에 train()함수를 호출시켜 학습 모드로 바꿔 줍니다.
'경사'라고도 할 수 있는 그레디언트(Gradient)는 오차 함수가 최소점을 가진 곳의 방향 입니다.
매 epoch 마다 우리는 새로운 그레디언트 값을 계산할 것이기 때문에 zero_grad()함수를 통해 그레디언트 값을 0으로 정의해 주겠습니다.

```python
    model.train()
    optimizer.zero_grad()
```

이미 생성한 모델에 학습데이터를 입력시켜 결과값을 계산합니다.
여기서 잠깐, 신경망 객체 속에 정의된 forward() 함수가 곧 신경망의 결과값을 내는 함수인 것은 맞지만, torch.nn.module이 forward() 함수 호출을 대신해줘 우리가 직접 호출할 필요는 없습니다.

```python
    train_output = model(x_tra) #torch.nn.module 을 통해서 forward()호출
```

신경망의 결과값의 차원을 레이블의 차원과 같도록 만들어 주고 오차를 계산합니다.

```python
    train_loss = criterion(train_output.squeeze(), y_tra)
```

학습이 잘 되는지 확인하기 위해 100 epoch마다 오차를 출력하도록 설정하겠습니다.

```python
	if epoch % 100 == 0:
		print('Train loss at ', epoch, 'is ', train_loss.item())
```

그 다음단계는 오차함수를 가중치 값들로 미분하여 오차함수의 최소점의 방향, 즉 그레디언트(Gradient)를 구하고 그 방향으로 모델을 러닝레이트 만큼 이동시키는 것입니다.

```python
    train_loss.backward()
    optimizer.step()
```

위 코드를 실행시켜 보면 오차값이 점점 줄어드는 것을 보실 수 있습니다.

```python
Train loss at  0 is  0.7301096916198730
Train loss at  100 is  0.6517783403396606
Train loss at  200 is  0.5854113101959229
Train loss at  300 is  0.519926130771637
Train loss at  400 is  0.4684883952140808
Train loss at  500 is  0.42419689893722534
Train loss at  600 is  0.3720306158065796
Train loss at  700 is  0.3115468919277191
Train loss at  800 is  0.25684845447540283
Train loss at  900 is  0.2133386880159378
```

바야흐로 우리의 첫 인공신경망 학습이 끝났습니다. 이제 학습된 신경망의 퍼포먼스를 시험할 차례입니다.
모델을 평가 모드(evaluation mode)로 바꿔 주고 실험데이터인 x_tes, y_tes를 이용해 오차값을 구해보겠습니다.

```python
model.eval()
test_loss_before = criterion(torch.squeeze(model(x_tes) ), y_tes)
print('Before Training, test loss is ', test_loss_before.item())
```

학습을 하기 전과 비교했을때 현저하게 줄어든 오차값을 확인 하실 수 있습니다.

```python
After Training, test loss is  0.20166122913360596
```

지금까지 인공신경망을 구현하고 학습시켜 보았습니다.
이제 학습된 모델을 .pt 파일로 저장해 보겠습니다.

```python
torch.save(model.state_dict(), './model.pt')
```

위 코드를 실행하고 나면 학습된 신경망의 가중치를 내포하는 model.pt 라는 파일이 생성됩니다. 아래 코드처럼 새로운 신경망 객체에 model.pt 속의 가중치값을 입력시키는 것 또한 가능합니다.

```python
new_model = Feed_forward_nn(2, 5)
new_model.load_state_dict(torch.load('./model.pt'))
new_model.eval()
print(new_model(torch.FloatTensor([-1,1])).item() )
```

여담으로 벡터 [-1,1]을 학습하고 저장된 모델에 입력시켰을 때 레이블이 1일 확률은 90% 이상이 나왔습니다.
우리의 첫번째 신경망 모델은 이제 꽤 믿을만한 분류 작업이 가능하게 된 것입니다.

```python
벡터 [-1,1]이 레이블 1 을 가질 확률은  0.9407910108566284
```

### 전체 코드

```python
import torch
import numpy
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plot
import torch.nn.functional as F

def label_map(y_, from_, to_):
    y = numpy.copy(y_)
    for f in from_:
        y[y_ == f] = to_
    return y
  
n_dim = 2
x_tra, y_tra = make_blobs(n_samples=80, n_features=n_dim, centers=[[1,1],[-1,-1],[1,-1],[-1,1]], shuffle=True, cluster_std=0.3)
x_tes, y_tes = make_blobs(n_samples=20, n_features=n_dim, centers=[[1,1],[-1,-1],[1,-1],[-1,1]], shuffle=True, cluster_std=0.3)
y_tra = label_map(y_tra, [0, 1], 0)
y_tra = label_map(y_tra, [2, 3], 1)
y_tes = label_map(y_tes, [0, 1], 0)
y_tes = label_map(y_tes, [2, 3], 1)

def vis_data(x,y = None, c = 'r'):
	if y is None:
		y = [None] * len(x)
	for x_, y_ in zip(x,y):
		if y_ is None:
			plot.plot(x_[0], x_[1], '*',markerfacecolor='none', markeredgecolor=c)
		else:
			plot.plot(x_[0], x_[1], c+'o' if y_ == 0 else c+'+')

plot.figure()
vis_data(x_tra, y_tra, c='r')
plot.show()

x_tra = torch.FloatTensor(x_tra)
x_tes = torch.FloatTensor(x_tes)
y_tra = torch.FloatTensor(y_tra)
y_tes = torch.FloatTensor(y_tes)

class Feed_forward_nn(torch.nn.Module):
		def __init__(self, input_size, hidden_size):
			super(Feed_forward_nn, self).__init__()
			self.input_size = input_size
			self.hidden_size  = hidden_size
			self.linear_1 = torch.nn.Linear(self.input_size, self.hidden_size)
			self.relu = torch.nn.ReLU()
			self.linear_2 = torch.nn.Linear(self.hidden_size, 1)
			self.sigmoid = torch.nn.Sigmoid()
		def forward(self, input_tensor):
			linear1 = self.linear_1(input_tensor)
			relu = self.relu(linear1)
			linear2 = self.linear_2(relu)
			output = self.sigmoid(linear2)
			return output

model = Feed_forward_nn(2, 5)
learning_rate = 0.03
criterion = torch.nn.BCELoss()
epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

model.eval()
test_loss_before =  criterion(model(x_tes).squeeze(), y_tes)
print('Before Training, test loss is ', test_loss_before.item())

for epoch in range(epochs):
	model.train()
	optimizer.zero_grad()
	train_output = model(x_tra)
	train_loss = criterion(train_output.squeeze(), y_tra)
	if epoch % 100 == 0:
		print('Train loss at ', epoch, 'is ', train_loss.item())
	train_loss.backward()
	optimizer.step()

model.eval()
test_loss = criterion(model(x_tes).squeeze(), y_tes) 
print('After Training, test loss is ', test_loss.item())

torch.save(model.state_dict(), './model.pt')
new_model = Feed_forward_nn(2, 5)
new_model.load_state_dict(torch.load('./model.pt'))
new_model.eval()
print(new_model(torch.FloatTensor([-1,1])).item() )
```
